In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation

import os
import pygaps


mpl.use('TkAgg')
plt.rcParams.update({'font.size': 14})
#plt.rcParams.update({'font.family': 'Times New Roman'})

class Generator:
    def __init__(self, path_s, path_d, path_p_s, path_p_d, path_a):
        with open(path_s, 'rb') as f:
            self.data_sorb = np.load(f)
        with open(path_d, 'rb') as f:
            self.data_desorb = np.load(f, allow_pickle=True)
        with open(path_p_d, 'rb') as f:
            self.pressures_d = np.load(f)
            self.pressures_d_current = self.pressures_d
        with open(path_p_s, 'rb') as f:
            self.pressures_s = np.load(f)
        with open(path_a, 'rb') as f:
            self.a_array = np.load(f)
        
        self.pore_distribution = None
        self.n_s = np.zeros(len(self.pressures_s))
        self.n_d = np.zeros(len(self.pressures_d))

    
    def generate_pore_distribution(self, sigma1, sigma2, d0_1, d0_2, a=1):
        pore_distribution1 = (1 / sigma1) * np.exp(-np.power((self.a_array - d0_1), 2) / (2 * sigma1 ** 2))
        pore_distribution1 /= max(pore_distribution1)
        pore_distribution2 = (1 / sigma2) * np.exp(-np.power((self.a_array - d0_2), 2) / (2 * sigma2 ** 2))
        pore_distribution2 /= max(pore_distribution2)
        self.pore_distribution = pore_distribution1*a + pore_distribution2
        self.pore_distribution /= max(self.pore_distribution)

    
    def calculate_isotherms_from_new_kernel(self):
        self.n_s = np.zeros(len(self.pressures_s))
        self.n_d = np.zeros(len(self.pressures_d))
        for p_i in range(len(self.pressures_s)):
            for d_i in range(len(self.pore_distribution)):
                self.n_s[p_i] += self.pore_distribution[d_i] * self.data_sorb[d_i][p_i]
                
        for p_i in range(len(self.pressures_d)):
            for d_i in range(len(self.pore_distribution)):
                if not np.isnan(self.data_desorb[d_i][p_i]):
                    self.n_d[p_i] += self.pore_distribution[d_i] * self.data_desorb[d_i][p_i]


    def normalize_data(self):
        self.n_s = self.n_s / self.n_s.max()
        self.n_d = self.n_d / self.n_d.max()
        
    def interp_desorption(self):
        self.n_d = np.interp(self.pressures_s, self.pressures_d, self.n_d)
        self.pressures_d_current = self.pressures_s

    
    def plot_isotherm(self):
        fig, axs = plt.subplots(2, 1)
        axs[0].plot(self.pressures_s, self.n_s, marker=".", label="Сорбция")
        axs[0].plot(self.pressures_d_current, self.n_d, marker=".", label="Десорбция")
        axs[0].set_xlabel("Давление")
        axs[0].set_ylabel("Величина адсорбции")
        axs[1].plot(self.a_array, self.pore_distribution, marker=".", label="Размер пор")
        axs[1].set_ylabel("Функция распределения")
        axs[1].set_xlabel("Размер пор (нм)")
        axs[0].legend()
        axs[1].legend()
        plt.show()
        return fig, axs
    
    def ani(self):
        fig, axs = plt.subplots(2, 1, figsize=(8,6))
        axs[0].set_ylim(0,2)
        self.generate_pore_distribution(sigma1=0.1, sigma2=2, d0_1=1, d0_2=10, a=0.1)
        self.calculate_isotherms_from_new_kernel()
        sorb_line, = axs[0].plot(self.pressures_s, self.n_s, marker=".", label="Сорбция")
        desorb_line, = axs[0].plot(self.pressures_d, self.n_d, marker=".", label="Десорбция")
        distr_line, = axs[1].plot(self.a_array, self.pore_distribution, marker=".")

        def animate(i):
            self.generate_pore_distribution(sigma1=0.1, sigma2=2, d0_1=1, d0_2=10, a=i)
            self.calculate_isotherms_from_new_kernel()
            self.normalize_data()
            sorb_line.set_ydata(self.n_s)  # update the data
            desorb_line.set_ydata(self.n_d)  # update the data
            distr_line.set_ydata(self.pore_distribution)  # update the data
            return sorb_line, desorb_line, distr_line
        
    
        anim = animation.FuncAnimation(fig, animate, np.linspace(0.1, 10, 250),
                                      interval=25, blit=False)
        writervideo = animation.FFMpegWriter(fps=30) 
        anim.save("anim.mp4", writer=writervideo)
        plt.show()


    def generate_data_set(self):
        d0_2_range = np.linspace(0.5, 10, 20)
        a_range = np.linspace(0.5, 2, 10)
        i = 0
        for a in a_range:
                for d0_2 in d0_2_range:
                    i+=1
                    self.generate_pore_distribution(sigma1=0.1, sigma2=2, d0_1=1, d0_2=d0_2, a=a)
                    self.calculate_isotherms_from_new_kernel()
                    self.interp_desorption()
                    # 
                    # 
                    # point_isotherm = pygaps.PointIsotherm(
                    # # First the pandas.DataFrame
                    # isotherm_data=pd.DataFrame({
                    #     'pressure' : np.concatenate((self.pressures_s, self.pressures_d), axis = 0),             # required
                    #     'loading' :  np.concatenate((self.n_s, self.n_d), axis = 0),              # required
                    # }),
                    #     material='carbon',              # Required
                    #     adsorbate='nitrogen',           # Required
                    #     temperature=77, 
                    #     
                    #     loading_key='loading',          # The loading column
                    #     pressure_key='pressure',        # The pressure column
                    #     d0_1 = d0_1,
                    #     d0_2 = d0_2,
                    #     
                    # )
                    # 
                    # point_isotherm.to_aif(f"test_set/{i}.aif")
                    np.savez(f"data/test_carbon/{i}", a = self.pore_distribution, d0_2 = d0_2, n_s = gen.n_s, n_d = gen.n_d)
    
    
    def save_isotherm_and_distribution(self, path):
        np.savez(path, n_s = gen.n_s, n_d = gen.n_d, distr = self.pore_distribution)
        


In [3]:
gen = Generator(path_s="data/kernel_generated2/Kernel_s_Carbon-loc-isoth-N2.xlsx.npy",
                        path_d="data/kernel_generated2/Kernel_d_Carbon-loc-isoth-N2.xlsx.npy",
                        path_p_d="data/kernel_generated2/Pressure_d_Carbon-loc-isoth-N2.xlsx.npy",
                        path_p_s="data/kernel_generated2/Pressure_s_Carbon-loc-isoth-N2.xlsx.npy",
                        path_a="data/kernel_generated2/Size_Carbon-loc-isoth-N2.xlsx.npy"
                )
gen.generate_data_set()
# gen.generate_pore_distribution(sigma1=1, sigma2=2, d0_1=1, d0_2=20, a=1)
# gen.calculate_isotherms_from_new_kernel()
# gen.interp_desorption()
# _ = gen.plot_isotherm()
# gen.save_isotherm_and_distribution("data/inverse problem/1.npz")
# 
# gen.ani()

In [ ]:
from matplotlib.widgets import Slider
 
gen2.generate_pore_distribution(sigma1=0.1, sigma2=1, d0_1=1, d0_2=10, path_a="data/kernel_generated2/Size_Carbon-loc-isoth-N2.xlsx.npy")
gen2.calculate_isotherms_from_new_kernel(path_s="data/kernel_generated2/Kernel_s_Carbon-loc-isoth-N2.xlsx.npy",
                                           path_d="data/kernel_generated2/Kernel_d_Carbon-loc-isoth-N2.xlsx.npy",
                                         path_p="data/kernel_generated2/Pressure_d_Carbon-loc-isoth-N2.xlsx.npy")
gen2.interp_desorption()
 
fig, ax = plt.subplots()
fig.subplots_adjust(bottom=0.2)
l, = ax.plot(gen2.pressures_s, gen2.n_s, marker=".")
 
def onChange(value):
    gen2.generate_pore_distribution(sigma1=0.1, sigma2=1, d0_1=1, d0_2=value, path_a="data/kernel_generated2/Size_Carbon-loc-isoth-N2.xlsx.npy")
    gen2.calculate_isotherms_from_new_kernel(path_s="data/kernel_generated2/Kernel_s_Carbon-loc-isoth-N2.xlsx.npy",
                                           path_d="data/kernel_generated2/Kernel_d_Carbon-loc-isoth-N2.xlsx.npy",
                                         path_p="data/kernel_generated2/Pressure_d_Carbon-loc-isoth-N2.xlsx.npy")
    gen2.interp_desorption()
    l.set_ydata(gen2.n_s)
    fig.canvas.draw_idle()
    ax.set_ylim(0, 1.1*max(gen2.n_s))
 
slideraxis = fig.add_axes([0.25, 0.1, 0.65, 0.03])
slider = Slider(slideraxis, label='Frequency [Hz]',
                valmin=0, valmax=10, valinit=5)
slider.on_changed(onChange)
plt.show()

In [ ]:
with open('new_data/new_a_array_s.npy', 'rb') as f:
    new_a_array_s = np.load(f)
with open('new_data/new_a_array_d.npy', 'rb') as f:
    new_a_array_d = np.load(f)

with open('new_data/new_isotherms_s.npy', 'rb') as f:
    data_sorb = np.load(f)
with open('new_data/new_isotherms_d.npy', 'rb') as f:
    data_desorb = np.load(f)

In [ ]:
df1 = pd.DataFrame(data_sorb,
                   columns = gen.pressures_s,
                   index=new_a_array_s).T

df2 = pd.DataFrame(data_desorb,
                   columns = gen.pressures_s,
                   index=new_a_array_s).T


with pd.ExcelWriter('newT.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='sorb')
    df2.to_excel(writer, sheet_name='desorb')

In [ ]:
def get_k_b(x1, y1, x2, y2):
        k = (y1 - y2) / (x1 - x2)
        b = y1 - k*x1
        return k, b

def find_nearests(a, n):
    value1 = a[np.argsort(np.abs(a-n))[0]]
    index1 = np.where(a == value1)[0]
    if value1 > n:
        index2 = index1 - 1
    else:
        index2 = index1 + 1
    return index1, index2

dataframe_sorb = pd.read_excel('SBA-15.xlsx', header=None, sheet_name="SBA-15")
# plt.plot(exp_p, exp_sorb, marker=".")
# plt.show()
exp_data = dataframe_sorb.to_numpy()
exp_p = exp_data[:, 3][1:75]
exp_sorb = exp_data[:, 4][1:75]
new_exp_sorb = np.empty(gen.pressures_s.shape)
# exp_scale = np.max(new_exp_sorb)
# new_exp_sorb = new_exp_sorb / exp_scale

for i in range(len(gen.pressures_s)):
    index1, index2 = find_nearests(exp_p, gen.pressures_s[i])
    x1 = exp_p[index1]
    x2 = exp_p[index2]
    y1 = exp_sorb[index1]
    y2 = exp_sorb[index2]
    k, b = get_k_b(x1, y1, x2, y2)
    new_exp_sorb[i] = k * gen.pressures_s[i] + b

In [ ]:
exp_scale = np.max(new_exp_sorb)
new_exp_sorb = new_exp_sorb / exp_scale

In [ ]:
plt.plot(exp_p, exp_sorb / exp_scale, marker=".")
plt.plot(gen.pressures_s, new_exp_sorb, marker=".")
plt.show()

In [ ]:
delta_d = new_a_array_s[1:] - new_a_array_s[0:1] 
delta_d[0], new_a_array_s[0], new_a_array_s[1]

In [ ]:
# нормирование
p_len = data_sorb.shape[1]
d_len = data_sorb.shape[0]
norm = np.zeros(p_len)
data_sorb_normed = np.zeros(data_sorb.shape)
nfit = np.zeros(p_len)
f_k = np.ones(d_len)
f_k_pr = np.ones(d_len)


# for k in range(d_len):
#     for i in range(p_len):
#         norm[k] += data_sorb[k][i]

for j in range(p_len):
    for i in range(d_len):
        norm[j] += data_sorb[i][j]


for j in range(d_len):
    for i in range(p_len):
        if norm[i] != 0:
            data_sorb_normed[j][i] = data_sorb[j][i] / norm[i]
        else:
            data_sorb_normed[j][i] = 0        

for i in range(p_len):
    for k in range(d_len):
        nfit[i]=data_sorb_normed[k][i]*f_k[k]


In [ ]:
nfit

In [ ]:
for step in range(5):     
        for i in range(p_len):
            sum = 0
            for k in range(d_len - 1):
                sum += data_sorb_normed[k][i]*f_k[k] * delta_d[k]
            nfit[i] = sum

        for j in range(d_len - 1):
            sum = 0
            for i in range(p_len):
                sum += nfit[i] / new_exp_sorb[i] * data_sorb_normed[j][i] * delta_d[k]
            f_k[j] = f_k_pr[j] * sum
            f_k_pr[j] = f_k[j]
        
    
        
        

In [ ]:
f_k

In [ ]:
plt.plot(gen.pressures_s, nfit, marker=".")
plt.show()

In [ ]:
plt.plot(new_a_array_s, f_k, marker=".")
plt.show()